In [1]:
import hashlib, re, time
import numpy as np
import pandas as pd

time_start = time.time()

raw_csv = pd.read_csv('raw_data.csv')
max_colname_len = max(
    [len(col) for col in raw_csv.columns.values]
)
for col in raw_csv.columns.values:
    print('{:{}}<{}>'.format(
        col, max_colname_len+2, raw_csv[col].dtype)
    )

Bird Species  <object>
Distance Bin  <object>
Date          <object>
Time          <object>
Coordinates   <object>
Location      <object>
Habitat Type  <object>
Time Period   <object>
Group Name    <object>
Remarks       <object>


In [ ]:
num_locations = len(set(raw_csv['Location']))
num_coords = len(set(raw_csv['Coordinates']))
print('Number of Unique Locations: {}'.format(num_locations))
print('Number of Unique Coordinates: {}'.format(num_coords))

Number of Unique Locations: 207
Number of Unique Coordinates: 270


**First of all, we need to be able to clearly distinguish survey sites.** As from the cell directly above, we cannot reliably use either the number of unique locations or coordinates in order to do this.

Instead, we will use both. Going through the list from the first to last row, _adjacent_ rows with identical locations **or** coordinate values will be grouped together as one survey site. Each survey site is identifed with a hash generated from the cocatenated string of its first row's location plus coordinate and group name.

In [ ]:
csv_unique_sites = pd.DataFrame()
old_row = None
current_hash = None

def get_loc_hash(row):
    '''@returns {str} Location Hash'''
    combined_in_bytes = (
        row['Coordinates'] + row['Location'] + row['Group Name']
    ).encode()
    loc_hash = hashlib.md5(combined_in_bytes).hexdigest()
    return loc_hash

def get_new_row(row, loc_hash):
    '''@returns {dict} Dictionary containing values for new row'''
    new_row = dict(row)
    new_row.pop('Date')
    new_row['Location Hash'] = loc_hash
    return new_row
    '''
    return {
        'Bird Species': row['Bird Species'],
        'Distance Bin': row['Distance Bin'],
        'Time': row['Time'],
        'Location Hash': loc_hash,
        'Coordinates': row['Coordinates'],
        'Location': row['Location'],
        'Habitat': row['Habitat Type'],
        'Time Period': row['Time Period'],
        'Group Name': row['Group Name'],
        'Remarks': row['Remarks']
    }'''
    
def is_same_site(row1, row2):
    if (row1 is None or row2 is None):
        return False
    else:
        is_same_coords = row1['Coordinates'] == row2['Coordinates']
        is_same_loc = row1['Location'] == row2['Location']
        return is_same_coords or is_same_loc

for _, row in raw_csv.iterrows():
    if not is_same_site(old_row, row):
        current_hash = get_loc_hash(row)
    csv_unique_sites = csv_unique_sites.append(
        get_new_row(row, current_hash),
        ignore_index=True)
    old_row = row

In [ ]:
num_locations = len(set(csv_unique_sites['Location Hash']))
print('Number of Unique Location Hashes: {}'.format(num_locations))

In [ ]:
csv_unique_sites.head(3)

In [ ]:
set(csv_unique_sites['Bird Species'])

Next, we want to standardise the formatting of each column, from the right to left. **Starting with the species names, we...**
1. Expand rows with multiple observations, e.g. Javan Myna x4
2. Replace observations with no clear IDs with NaN values
3. Standardise trailing white-spaces, spellings, and capitalisations

In [ ]:
better_names = pd.DataFrame()
contains_digits = re.compile('\d')
bad_idents = re.compile(
    '(\`|[Un]nknown|[Ss]uspected|\?|None|^\-$|'
    'sound|chweet|with| w |that|Large black bird|'
    '[Ss]potted [Pp]igeon)'
)

def process_row(row):
    '''@returns {tuple} <repeats: int, row: dict>'''
    row = dict(row)  # none-destructive
    # first, check if this row contains multiple sightings
    if contains_digits.search(row['Bird Species']):
        print('Has multiple: {}'.format(row['Bird Species']))
        repeats = contains_digits.search(row['Bird Species']).group(0)
        repeats = int(repeats)
        row['Bird Species'] = row['Bird Species'].strip()
        last_whitespace_i = row['Bird Species'].rfind(' ')
        row['Bird Species'] = row['Bird Species'][:last_whitespace_i]
    else:
        repeats = 1
    # then, standardise the species naming
    row['Bird Species'] = row['Bird Species'].strip().lower().capitalize()
    # next, check for bad IDs
    if bad_idents.search(row['Bird Species']):
        print('Is bad ID: {}'.format(row['Bird Species']))
        row['Bird Species'] = np.nan
    return (repeats, dict(row))
        
for _, row in csv_unique_sites.iterrows():
    repeats, new_row = process_row(row)
    for _ in range(repeats):
        better_names = better_names.append(new_row, ignore_index=True)

In [ ]:
sorted(
    [str(x) for x in list(set(better_names['Bird Species']))]
)  # str conversion because nan is a float, not comparable with str

Now, we standardise spellings...
```
Asian koele -> Asian koel
Crow -> Jungle crow
Javan mynah -> Javan myna
Koel -> Asian koel
Large-billed crow -> Jungle crow
Olive-winged bulbol -> Olive-winged bulbul
Olve-backed sunbird -> Olive-backed sunbird
Pink neck green pegion -> Pink-necked green pigeon
Pink neck green pigeon -> Pink-necked green pigeon
Rocked pigeon -> Rock pigeon
Swiflet -> Swiftlet
Swiflets -> Swiftlet
Swiftlets -> Swiftlet
```

In [ ]:
renames = {
    'Asian koel': re.compile('(Asian koele|^Koel$)'),
    'Jungle crow': re.compile('(^Crow$|Large-billed crow)'),
    'Javan myna': re.compile('Javan mynah'),
    'Olive-winged bulbul': re.compile('Olive-winged bulbol'),
    'Olive-backed sunbird': re.compile('Olve-backed sunbird'),
    'Pink-necked green pigeon': re.compile('Pink neck green p[ei]g[ie]on'),
    'Rock pigeon': re.compile('Rocked pigeon'),
    'Swiftlet': re.compile('(Swiflet[s]|Swiftlets)')
}

for _, row in better_names.iterrows():
    for proper_name, has_bad_name in renames.items():
        try:
            if has_bad_name.search(row['Bird Species']):
                print('Caught: {} -> {}'.format(
                    row['Bird Species'], proper_name))
                row['Bird Species'] = proper_name
        except TypeError:
            pass  # because of nan values

In [ ]:
sorted(
    [str(x) for x in list(set(better_names['Bird Species']))]
)  # str conversion because nan is a float, not comparable with str

In [ ]:
set(better_names['Distance Bin'])

Next, we want to discount all birds in flight or heard but not seen. These observations will have a distance bin of `nan`

In [ ]:
bad_distance = re.compile(
    '([Hh]eard|[Ff]lyby|[Ff]light|Did not land|^\-$)'
)

for _, row in better_names.iterrows():
    try:
        if bad_distance.search(row['Distance Bin']):
            row['Distance Bin'] = np.nan
    except TypeError:
        pass  # once again, nan is a float and cannot be searched

In [ ]:
set(better_names['Distance Bin'])

In [ ]:
set(better_names['Time'])

Now, we want to standardise the times

In [ ]:
is_dd_space_dd = re.compile('^\d\d \d\d$')
def fix_dd_space_dd(timestr):
    return timestr.replace(' ', '')

is_dd_colon_dd = re.compile('^\d\d:\d\d$')
def fix_dd_colon_dd(timestr):
    return timestr.replace(':', '')

is_d_dot_d = re.compile('^\d\.\d$')
def fix_d_dot_d(timestr):
    timestr = '0{}0'.format(timestr)
    return timestr.replace('.', '')

is_d_colon_dd = re.compile('^\d:\d\d$')
def fix_d_colon_dd(timestr):
    timestr = '0{}'.format(timestr)
    return fix_dd_colon_dd(timestr)

is_d_dot_dd = re.compile('^\d\.\d\d$')
def fix_d_dot_dd(timestr):
    timestr = '0{}'.format(timestr)
    return timestr.replace('.', '')

is_ddd = re.compile('^\d\d\d$')
def fix_ddd(timestr):
    return '0{}'.format(timestr)

is_d = re.compile('^\d$')
def fix_d(timestr):
    return '0{}00'.format(timestr)

time_fixes = {
    is_dd_space_dd: fix_dd_space_dd,
    is_dd_colon_dd: fix_dd_colon_dd,
    is_d_dot_d:     fix_d_dot_d,
    is_d_colon_dd:  fix_d_colon_dd,
    is_d_dot_dd:    fix_d_dot_dd,
    is_ddd:         fix_ddd,
    is_d:           fix_d
}

for _, row in better_names.iterrows():
    for bad_time_re, fixer in time_fixes.items():
        try:
            if bad_time_re.search(row['Time']):
                print('{} -> {}'.format(row['Time'], fixer(row['Time'])))
                row['Time'] = fixer(row['Time'])
        except TypeError:
            pass  # ignore nan values, as usual

In [ ]:
set(better_names['Time'])

Next up are the coordinates, which will be a bit challenging...

To start, we standardise the coordinates and locations of each unique survey site (identified by it's location hash, generated earlier). 

This fixes typos, and the cases where a user drag-copied location and coordinate cells, causing google sheets to increment the last character if it is a number.

In [ ]:
num_locations = len(set(better_names['Location']))
num_coords = len(set(better_names['Coordinates']))
num_unique_locations = len(set(better_names['Location Hash']))
print('Number of Unique Locations: {}'.format(num_locations))
print('Number of Unique Coordinates: {}'.format(num_coords))
print('Number of Unique Location Hashes: {}'.format(num_unique_locations))

In [ ]:
old_pointer = {'hash': '', 'coord': '', 'loc': ''}

for _, row in better_names.iterrows():
    if row['Location Hash'] != old_pointer['hash']:
        old_pointer['hash'] = row['Location Hash']
        old_pointer['coord'] = row['Coordinates']
        old_pointer['loc'] = row['Location']
    else:
        row['Coordinates'] = old_pointer['coord']
        row['Location'] = old_pointer['loc']

In [ ]:
num_locations = len(set(better_names['Location']))
num_coords = len(set(better_names['Coordinates']))
num_unique_locations = len(set(better_names['Location Hash']))
print('Number of Unique Locations: {}'.format(num_locations))
print('Number of Unique Coordinates: {}'.format(num_coords))
print('Number of Unique Location Hashes: {}'.format(num_unique_locations))

It seems there are two duplicate coordinates, and two pairs of duplicate locations (or a triplet of identical locations). Let's figure which those are.

In [ ]:
from collections import Counter

old_pointer = {'hash': '', 'coord': '', 'loc': ''}
locations = []
coords = []

for _, row in better_names.iterrows():
    if row['Location Hash'] != old_pointer['hash']:
        # add each new location to list
        locations.append(row['Location'])
        coords.append(row['Coordinates'])
        # reset the old_pointer
        old_pointer['hash'] = row['Location Hash']
        old_pointer['loc'] = row['Location']
        old_pointer['coord'] = row['Coordinates']
        
location_counts = Counter(locations)
coord_counts = Counter(coords)

for loc, reps in location_counts.items():
    if reps > 1:
        print('Location \'{}\' occured {} times'.format(
            loc, reps
        ))

for coord, reps in coord_counts.items():
    if reps > 1:
        print('Coordinate {} occured {} times'.format(
            coord, reps
        ))

Repeat of PGP and RVRC locations are acceptable. 

The coordinate duplicates are for town green. These two survey sites were remarked: _"Town Green data points were very close to one another - there might be potential recounting of bird species."_

Now, to standardise the coordinates. We need to,

1. Remove verbose labellings e.g. 'Lat', 'Lon'
2. Convert DMS notation to Decimal degrees

In [ ]:
set(better_names['Coordinates'])

In [ ]:
is_word = re.compile('[a-zA-Z]{2,}')

for _, row in better_names.iterrows():
    coord = row['Coordinates']
    coord = is_word.sub('', coord)
    coord = coord.replace('|', ',')
    coord = coord.replace(':', '')
    coord = coord.replace(' ', '')
    row['Coordinates'] = coord

In [ ]:
set(better_names['Coordinates'])

Now,

$$\mathrm{D_{dec}} = 
    \mathrm{D} + \frac{\mathrm{M}}{60} + \frac{\mathrm{S}}{3600}$$
    
Note that some seconds are recorded to 0 d.p. precision, others to 1 d.p.

$$ 1'' = 0.0002\bar{7}^{\ \circ} $$
$$ 0.1'' = 0.00002\bar{7}^{\ \circ} \mathrm{(6\ d.p.)}$$

Therefore, we round off to 6 d.p., _if_ the decimal degrees exceeds 6 d.p.

In [ ]:
def get_dms(dms_coords):
    digits = re.compile('\d+\.?\d*')
    return tuple(map(float, digits.findall(dms_coords)))

def get_decimal(dms_xs):
    lat_d, lat_m, lat_s = dms_xs[:3]
    lon_d, lon_m, lon_s = dms_xs[3:]
    lat = lat_d + lat_m/60 + lat_s/3600
    lon = lon_d + lon_m/60 + lon_s/3600
    lat, lon = round(lat, 6), round(lon, 6)
    return '{},{}'.format(lat, lon)

for _, row in better_names.iterrows():
    if 'N' in row['Coordinates'] or 'E' in row['Coordinates']:
        dms = get_dms(row['Coordinates'])
        decimal = get_decimal(dms)
        row['Coordinates'] = decimal

Finally, to wrap up the coordinate clean-up, we split lat and lon into two separate columns.

In [ ]:
latlon_df = pd.DataFrame()

for _, row in better_names.iterrows():
    row = dict(row)
    coords = row.pop('Coordinates')
    lat, lon = coords.split(',')
    row['Latitude'] = lat
    row['Longitude'] = lon
    latlon_df = latlon_df.append(row, ignore_index=True)

In [ ]:
latlon_df.head(3)

In [ ]:
tuple(latlon_df.columns.values)

In [ ]:
set(latlon_df['Habitat Type'])

In [ ]:
for _, row in latlon_df.iterrows():
    try:
        if 'NIL' in row['Habitat Type']:
            print('Habitat Type: {}; Distance Bin: {}'.format(
                row['Habitat Type'], row['Distance Bin']
            ))
    except TypeError:
        pass

Next up we have the 'Habitat Type' column. This column is largely useless, though some groups have recorded 'inflight' or 'heard but not seen' data here---while leaving legitimate values in the distance bin.

Since we are using `nan` values in the distance bin to signify inflights and seen-but-not-heards, we need to do a bit of modifying. Afterwards, the habitat column can be ignored.

In [ ]:
for _, row in latlon_df.iterrows():
    try:
        if 'NIL' in row['Habitat Type']:
            row['Distance Bin'] = np.nan
    except TypeError:
        pass  # if row['Habitat Type'] == np.nan

In [ ]:
# check to see the changes...

for _, row in latlon_df.iterrows():
    try:
        if 'NIL' in row['Habitat Type']:
            print('Habitat Type: {}; Distance Bin: {}'.format(
                row['Habitat Type'], row['Distance Bin']
            ))
    except TypeError:
        pass

Similarly, we inspect the remarks to see if we might need to modify any rows based on the information within.

In [ ]:
for _, row in latlon_df.iterrows():
    # np.nan is of type float
    if type(row['Remarks']) is not float:
        print(row)

Visual inspection of the remarks column shows that some heard-but-not-seen and bad identifications have not had their distance bins or bird species columns respectively set to `nan`. Namely, they are:

```
Sound heard but not observed @ b39d42986a259bde849d4bd0ef74c4df  
Heard sound but not observed, 180 degree view @ 332d8684a20391d12ae81d9d8e0aa1c9  
Identity cannot be confirmed, but features see... @ c79289b16fa0a087f2ef63771d8dfe73
```

In [ ]:
for _, row in latlon_df.iterrows():
    if (type(row['Remarks']) == str and (
        (
            'Sound heard but not observed' in row['Remarks'] or
            'Heard sound but not observed' in row['Remarks']
        ))):
        print('Caught: {}'.format(row['Remarks']))
        row['Distance Bin'] = np.nan
    elif (type(row['Remarks']) == str and
         'Identity cannot be confirmed' in row['Remarks']):
        row['Bird Species'] == np.nan
        print('Caught: {}'.format(row['Remarks']))

Lastly, we standardise the time period. Since all surveys are 10 minutes long, we can truncate the later bound of the time periods.

In [ ]:
set(latlon_df['Time Period'])

In [ ]:
# we can reuse the functions from time_fixes from when
#     we sanitised the column 'Time' earlier. Hoever,
#     there is a new format HH.MM to include.
time_fixes[re.compile('^\d\d\.\d\d$')] = (
    lambda timestr: timestr.replace('.', '')
)

for _, row in latlon_df.iterrows():
    time_ = row['Time Period'].split('-')[0].strip()
    # we can reuse the functions earlier from the column 'Time'
    for bad_time_re, fixer in time_fixes.items():
        if bad_time_re.search(time_):
            time_ = fixer(time_)
    row['Time Period'] = time_

In [ ]:
set(latlon_df['Time Period'])

In [ ]:
for name in latlon_df.columns.values:
    print(set(latlon_df[name]))

Before we save to csv, reorder the columns for presentation's sake

In [ ]:
latlon_df.columns.tolist()

In [ ]:
latlon_df = latlon_df[[
    'Bird Species',
    'Distance Bin',
    'Time',
    'Location Hash',
    'Latitude',
    'Longitude',
    'Time Period',
    'Group Name',
    'Location',
    'Habitat Type',
    'Remarks'
]]

In [ ]:
latlon_df.to_csv('data.csv')
print('Time taken: {:.1f}s'.format(time.time() - time_start))